## An example workflow for schism

This Notebook assumes locally stored datasets 

In [ ]:
#to use the full width of the browser window uncomment the code below and execute the cell
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pyPoseidon
import pyPoseidon.model as pmodel
import datetime
import pandas as pd
from pyPoseidon.utils import data,point

In [ ]:
pyPoseidon.__version__

In [ ]:
import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs

In [ ]:
from hvplot import xarray, pandas

### Define case

In [ ]:
#define in a dictionary the properties of the model..
dic={'solver':'schism',
     'grid_file': '/Users/brey/SCHISM/test_grid/hgrid.gr3',
     'config_file': '/Users/brey/SCHISM/test/param.in',
     'manning':.12,
     'windrot':0.00001,
     'tag':'eur',
     'start_date':'2018-9-1 0:0:0',
     'time_frame':'12H',
     'ft1':0,
     'ft2':11,
#     'end_date':'2013-2-5 0:0:0',
     'meteo':'ecmwf_oper',
#     'dem': 'gebco',
#     'dpath' : '/Users/brey/DATA/GEBCO_2014_2D.nc',
     'epath':'/Users/brey/SCHISM/v5.6.1/src/schism.ex_VL', #solver executables
     'ncores': 4 , #number of cores
     'rpath':'/Users/brey/Downloads/EUR/SCHISM/20180901.00/', #location of calc folder
     'conda_bin':'/Users/brey/miniconda3/bin/', #conda path
     'conda_env': 'mpi3', # optional conda env for the solver
    }

#### Local operational ECMWF files

In [ ]:
if 'time_frame' in dic.keys(): 
    end_date= pd.to_datetime(dic['start_date']) + pd.to_timedelta(dic['time_frame'])
    dic.update({'end_date':end_date.strftime(format='%Y-%m-%d %H:%M:%S')})

In [ ]:
dic['end_date']

In [ ]:
# FOR FORECAST - SINGLE FILE
dr = [pd.to_datetime(dic['start_date'])]
dr

In [ ]:
#FOR USING THE OPTIMUM METEO
dr = pd.date_range(dic['start_date'],dic['end_date'], freq='12H')
dr

In [ ]:
#creating a sequence of folder from which we read the meteo.

PATH='/Users/brey/DATA/OPER/'#Path to meteo files
#PATH = '/Volumes/data/projects/CRITECH/meteo/ECMWF/operational/'
folders = [datetime.datetime.strftime(x, '%Y%m%d.%H') for x in dr]
meteo_files = [PATH + datetime.datetime.strftime(x, '%Y%m%d.%H')+'.uvp_72.grib' for x in dr]
meteo_files

#### update dictionary

In [ ]:
dic.update({'mpaths':meteo_files, 'engine':'cfgrib', 'combine':True})

## Initialize

In [ ]:
#initialize a model
b = pmodel.model(**dic)

### set it up

In [ ]:
b.set(**dic) #set it up 

In [ ]:
b.impl.grid.impl.Dataset['land boundary 1']

In [ ]:
#Change param variables accordingly 
b.impl.params.loc['rnday'] = .4
b.impl.params.loc['hotout'] = 1 # save hotstart
b.impl.params.loc['hotout_write'] = 108 # Every x timesteps
b.impl.params.loc['ihot']=0 # no hotstart 

In [ ]:
b.impl.meteo.impl.uvp.time

## Save to folder for execution 

In [ ]:
#set the run by saving the files
b.output()

In [ ]:
# save model info for further use
b.save()

In [ ]:
# save all matrices for further use
#b.pickle(path=path)

### execute

In [ ]:
#execute
b.run()

## Analysis of output 

In [ ]:
#%matplotlib inline

In [ ]:
otp = data(**dic)#,tag='eur')# optional tag for data selection ,sa_date='2010-2-1 5:0:0',se_date='2010-2-1 22:0:0') # optionally set range for tide gauge data acquisition 

In [ ]:
otp.impl.Dataset

In [ ]:
otp.impl.Dataset.splot.contourf('depth' )#,vmin=0, vmax=6452)

In [ ]:
otp.impl.Dataset.splot.grid()

In [ ]:
otp.impl.Dataset.splot.quiver('dahv',it=5,scale=0.5e-1)

### Visualize

In [ ]:
otp.impl.Dataset.splot.frames('elev',title='SSH')#,vmin=-.5, vmax=.5, nv=20)

In [ ]:
otp.impl.Dataset.splot.qframes('wind_speed',title='Vel',scale=30.)

## Observation points / Validation

In [ ]:
otp.impl.obs.locations

In [ ]:
# Get with index number
p = otp.impl.obs.iloc(2)
print (p.head())
print (p.tail())

In [ ]:
#get with Station Name  
p = otp.impl.obs.loc('Denmark - Hirtshals')
p.head()

In [ ]:
#plot
ax = p.plot()

### sample nearest point from simulation 

In [ ]:
 plat,plon = otp.impl.obs.locations.loc[2,['lat','lon']]

In [ ]:
ts = point(lon=plon,lat=plat,data=otp.impl.Dataset)
ts.stseries(var='elev')

In [ ]:
ts.elev

In [ ]:
ts.elev.plot()

In [ ]:
## Join the graphs
ax = p.plot(figsize=(14,10))
ts.elev.plot(ax=ax,color=['r'])
ax.legend(list(p.columns.values) + ['Schism'], loc='center left',bbox_to_anchor=(1.1, 0.5))